In [ ]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

In [ ]:
Q,R = qr(X)

In [ ]:
Q

In [ ]:
R

In [ ]:
Q'*Q

In [ ]:
# form data from noisy linear model
w♮ = randn(d)
y = X*w♮ + .1*randn(n);

In [ ]:
# solve least squares problem to estimate w
w = R \ (Q'*y)

In [ ]:
# how good is our estimate?
norm(w - w♮)

In [ ]:
# let's use the shorthand
w = X \ y
norm(w - w♮)